# Kvantni algoritmi 1

## Kvantna kola za računanje klasičnih funkcija

Obradićemo sada jednu bitnu klasu kvantnih algoritama: algoritmi koji izračunavaju klasične funkcije. Klasične funkcije su bitne i u kvantnom računarstvu, jer one su osnova nekih bitnih kvantnih algoritama, kao što je Šorov algoritam koji se zasniva na efikasnoj implementaciji kvantnog kola koje računa modularno stepenovanje broja.

Što se uvoda tiče i kvantnih sabirača, koristimo algoritme iz ove [knjige](https://mitpress.mit.edu/9780262526678/quantum-computing/). Nažalost, algoritmi koji rade modularne operacije iz iste knjige sadrže neotklonjive greške. Zbog toga ćemo pokušati da implementiramo svoje algoritme koji rade isti zadatak.

#### Glavna ideja:

Kvantna kola bez kvantnog merenja su *reverzibilna*, jer je unitarna transformacija invertibilna. Klasična kola u opštem slučaju nisu reverzibilna.

Reverzibilno klasično kolo koje ima $n$ bitova na ulazu i $n$ bitova na izlazu se može predstaviti kao bijekcija $f: \mathbb{Z}^n \xrightarrow{} \mathbb{Z}^n$, odnosno kao permutacija skupa $\mathbb{Z}^n$. Kada nam je ova funkcija poznata, kvantno kolo koje izračunava $f$ se može, na baznim vektorima $\ket{x}$, odrediti jednačinom:
\begin{align*}
\sqcup_f \ket{x} = \sqcup_f \ket{f(x)}.
\end{align*}

U slučaju da je $f: \mathbb{Z}^n \xrightarrow{} \mathbb{Z}^m$, za $n \neq m$, ovo klasično kolo možemo prevesti u reverzibilno klasično kolo definisanjem funkcije $g: \mathbb{Z}^{n + m} \xrightarrow{} \mathbb{Z}^{n + m}$:
\begin{align*}
g(x, y) = (x, y \oplus f(x)),
\end{align*} za sve $x \in \mathbb{Z}^n$ i $y \in \mathbb{Z}^m$, gde je $\oplus$ bitovski $XOR$.

Odavde sledi konstrukcija za kvantno kolo $\sqcup_f$ definisano jednačinama na baznim vektorima $\ket{x} \otimes \ket{y}$:
\begin{align*}
\sqcup_f \ket{x} \ket{y} = \ket{x} \ket{y \oplus f(x)}.
\end{align*}

### NOT:

Kvantno kolo $X$ je analog klasičnom kolu $NOT$. Zaista, bijekcija koja računa $NOT$ je $f(0) = 1$, $f(1) = 0$. Odavde je $\sqcup_f = X$.

### XOR:

Standardna verzija funkcije koja izračunava $XOR$ je $f(x, y) = x \oplus y$. Broj bitova na ulazu je $2$, a broj bitova na izlazu je $1$. Ako bismo pratili postupak koji smo dali kada je $n \neq m$, reverzibilno klasično kolo bi imalo $3$ bitova na ulazu i na izlazu. U ovom slučaju može i efikasnije, ako se uzme $f(x, y) = (x, x \oplus y)$, što daje $2$ bitova na ulazu i na izlazu. Dakle, $\sqcup_f$ je određeno jednačinama:
\begin{align*}
\sqcup_f \ket{x} \ket{y} = \ket{x} \ket{y \oplus x},
\end{align*} za sve bazne vektore $\ket{x}\ket{y}$. Odavde nije teško dokazati da je $\sqcup_f = CNOT$.

### AND:

Standardna verzija funkcije koja izračunava $AND$ je $f(x, y) = 1$ ako i samo ako je $x = 1$ i $y = 1$. Reverzibilna verzija se može konstruisati postupkom koji smo dali iznad, čime dobijamo $g(x, y, z) = (x, y, z \oplus f(x, y))$. Odavde se lako dobija $\sqcup_f$. Primetimo da će vrednost $f(x, y)$ biti $1$ ako i samo ako su $x$ i $y$ jednaki $1$. U tom slučaju se vrednost od $z$ invertuje, a inače vrednost trećeg bita na izlazu ostaje ista. Na kvantnom računaru ovo ponašanje na baznim vektorima ima $CCNOT$. Dakle, $\sqcup_f = CCNOT$.

### NAND:

$NAND$ predstavlja kompoziciju $AND$ i $NOT$. Nakon dobijanja $\ket{x}\ket{y}\ket{z \otimes f(x, y)}$, primenom $X$ na trećem kubitu se dobija $\ket{x}\ket{y}\ket{z \otimes NOT(f(x, y))} = \ket{x}\ket{y}\ket{z \otimes NAND(x, y)}$. Kvantno kolo koje implementira ovo je $(I \otimes I \otimes X)CCNOT$.

### Alternativna konstrukcija:

Moguće je sve četiri prethodne operacije objediniti jednom operacijom $CCNOT$, ako se argumenti dobro nameste. To je zahvaljujući tome što važe naredne jednakosti:
\begin{align*}
CCNOT \ket{11}\ket{x} &= \ket{11}\ket{NOT(x)}, \\
CCNOT \ket{1}\ket{x}\ket{y} &= \ket{1}\ket{x}\ket{x \oplus y}, \\
CCNOT \ket{x}\ket{y}\ket{0} &= \ket{x}\ket{y}\ket{AND(x, y)}, \\
CCNOT \ket{x}\ket{y}\ket{1} &= \ket{x}\ket{y}\ket{NAND(x, y)},
\end{align*} za bazne vektore $\ket{x}$ i $\ket{y}$.

### Sabirač:

#### Sabiranje dva bita:

Ova konstrukcija je elementarna i biće korišćena da definiše složenije sabirače. Na ulazu se nalaze bitovi $a$ i $b$ koje treba sabrati i prenosni bit $c$. Dakle, rezultat treba da bude $(a +_2 b +_2 c)$. Na klasičnom računaru, ovo ponašanje definiše funkcija $f(c, a, b) = (a \oplus b \oplus c)$. Da bi ta funkcija postala reverzibilna, možemo definisati $g(c, a, b) = (c, a, c \oplus a \oplus b)$. Pomoću $CNOT$ kvantnih kola, možemo na trećem kubitu dodavati $XOR$ operacije, kao što vidimo u kodu ispod.

##### Sabiranje dva bita u Q#

In [1]:
import qsharp

In [2]:
%%qsharp

open Microsoft.Quantum.Diagnostics;

operation BitAdd(c : Qubit, a : Qubit, b : Qubit) : Unit is Adj + Ctl {
    CNOT(a, b);
    CNOT(c, b);
}

##### Računanje prenosa:

Prilikom sabiranja bitova $a$, $b$ i $c$, do prenosa na sledeću poziciju dolazi ako su bar dva bita vrednosti $1$. Izraz koji računa da li je došlo do prenosa je $(a \land b) \oplus (c \land (a \oplus b))$. Ovo se sada može lako implementirati kao kvantno kolo sa četiri kubita.

##### Računanje prenosa u Q#:

In [3]:
%%qsharp

operation BitCarry(c : Qubit, a : Qubit, b : Qubit, r : Qubit) : Unit is Adj + Ctl {
    // smeštamo AND(a, b) u r
    CCNOT(a, b, r);
    // smeštamo XOR(a, b) u b
    CNOT(a, b);
    // u r je AND(a, b)
    // ako je AND(c, XOR(a, b)), onda treba r da invertujemo
    // inače ne treba ništa da radimo
    // trenutno je u b XOR(a, b)
    CCNOT(c, b, r);
    // u r je sada XOR((XOR(a, b) AND c), AND(a, b))
    // vraćamo b na staru vrednost
    CNOT(a, b);
    // na kraju je izlaz |a>|b>|c>|XOR((XOR(a, b) AND c), AND(a, b))>
}

#### Višebitni sabirač:

Pokazaćemo kako se može sabiranje dva $n$-bitna broja izvesti na kvantnom računaru. Kvantno kolo koje to izvodi treba da zadovoljava sledeću jednakost:
\begin{align*}
Add \ket{c}\ket{a}\ket{b} = \ket{c}\ket{a}\ket{(a +_{2N} b +_{2N} c)},
\end{align*} za sve bazne vektore $\ket{a}\ket{b}\ket{c}$, gde je $N = 2^{n}$. Registar $b$ ima $n + 1$ bit, dok $a$ i $b$ sadrže po $n$ bitova. Početno stanje od $c$ je $\ket{0 \ldots 0}$, a $\ket{b} = \ket{0}\ket{b'}$.

Bez daljih objašnjenja, implementiraćemo kod iz knjige.

##### Višebitni sabirač u Q#:

In [4]:
%%qsharp

open Std.Arrays;

operation Add(c : Qubit[], a : Qubit[], b : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(c);
    if n == 1 {
        BitCarry(c[0], a[0], b[1], b[0]);
        BitAdd(c[0], a[0], b[1]);
    } else {
        BitCarry(c[n - 1], a[n - 1], b[n], c[n - 2]);
        Add(Most(c), Most(a), Most(b));
        Adjoint BitCarry(c[n - 1], a[n - 1], b[n], c[n - 2]);
        BitAdd(c[n - 1], a[n - 1], b[n]);
    }
}

#### Bibliotečka funkcija za računanje zbira u Q#:

U biblioteci **Std.Arithmetic**, nalazi se funkcija koja računa zbir dva broja na kvantnom računaru. Potpis te funkcije je **operation AddLE(xs : Qubit[], ys : Qubit[], zs : Qubit[]) : Unit is Adj**. Zvanična [dokumentacija](https://learn.microsoft.com/en-us/qsharp/api/qsharp-lang/std.arithmetic/addle) kaže da registri **xs**, **ys** i **zs** smeštaju najznačajniji bit na poslednje mesto. Takođe, **xs** i **ys** su iste dužine, a dužina od **zs** je barem onolika kolika je od **xs**. Pre izvršavanja, **zs** bi trebalo da bude u stanju $\ket{0 \ldots 0}$. Nakon izvršavanja **AddLE**, u **zs** će biti zbir **xs** i **ys**.

Osim ove, postoji u istoj biblioteci i **operation RippleCarryCGAddLE(xs : Qubit[], ys : Qubit[], zs : Qubit[]) : Unit is Adj**. Obe funkcije rade isto, samo što **RippleCarryCGAddLE** radi efikasnije od **AddLE**.

### Množenje na kvantnom računaru:

Ne postoji bibliotečka funkcija za računanje proizvoda u Q#. Ovde ćemo prikazati kako se može raditi množenje po modulu $2^n$, za neko $n$. Algoritam koji sada prikazujemo je predložila grupa autora u sledećem [radu](https://ieeexplore.ieee.org/document/9262868).

Na ulazu u kvantno kolo su nam dati $\ket{a}$, $\ket{b}$, $\ket{c_0}$, $\ket{c_1}$. Registri $a$, $b$, $c_0$ i $c_1$ su veličine $n$. Rezultat množenja je $a \cdot_N b$, gde je $N = 2^n$, i rezultat se smešta u registar $c_1$. Početno stanje registara $c_0$ i $c_1$ je $\ket{0 \ldots 0}$. Takođe, pošto je $\ket{c_0}$ u stanju $\ket{0 \ldots 0}$ pre i posle izvršavanja operacije, možemo da pretvorimo $c_0$ u pomoćne kubite.

In [5]:
%%qsharp

open Std.Arrays;
open Std.Arithmetic;

operation Multiply(a : Qubit[], b : Qubit[], c1 : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(a);
    use c0 = Qubit[n];
          
    // u registar c1 se smešta prvi parcijalni proizvod, odnosno a[0] * b
    for i in 0..n-1 {
        CCNOT(a[0], b[i], c1[i]);
    }

    for i in 1..n-1 {
        within {
            // u c0 smeštamo naredni parcijalni proizvod bez bitova posle n-tog
            for j in 0..n-1-i {
                CCNOT(a[i], b[j], c0[i + j]);
            }
        } apply {
            // ova funkcija efikasno dodaje vrednost od c0 na c1
            RippleCarryCGIncByLE(c0, c1);
        }
    }
}

Ovde smo prvi put primenili blokove **within** i **apply**. Dosta kvantnih algoritama se zasniva na izračunavanju oblika $V U V^{\dagger}$. Zbog toga je jedna od naredbi u Q# within-apply. U bloku **within** pišemo šta radi operacija $V$, a u bloku apply pišemo šta radimo operacija $U$. Pošto će operacija $V$ biti invertovana, bitno je da se u njoj primenjuju samo **Adj** operacije.

### Modularno sabiranje na kvantnom računaru

Osnovna ideja za realizaciju ove operacije je da se argumenti unitarne transformacije ograničene na potprostor generisan baznih vektorima koji odgovaraju brojevima manjim od $M$, gde je $M$ broj po čijem modulu računamo zbir. Ovo se može opisati narednom funkcijom:

\begin{align*}
f_M(x, y) &= (x, y), \text{za $x \geq M$ ili $y \geq M$}, \\
f_M(x, y) &= (x, x + y), \text{za $x, y < M$ i $x + y < M$}, \\
f_M(x, y) &= (x, x + y - M), \text{za $x, y < M$ i $x + y \geq M$}.
\end{align*}

Analizom po svim slučajevima, može se pokazati da je ova funkcija bijekcija.

Zbog jednostavnije implementacije, može se podrazumevati da $x, y < M$ i funkcija se implementirati slično kao u knjizi *Rieffel, Polak*.

#### Modularno sabiranje na kvantnom računaru

Implementacija u **Q#** se zasniva na dodavanju $a$ na $b$, nakon čega se oduzima $M$ od $b$. Dodatni kubit će postati $\ket{1}$ ako je došlo do prekoračenja, odnosno ako je $a + b < M$. U tom slučaju dodajemo $M$ na $b$. Dužina registara $a$, $b$ i $M$ je $n$ takvo da je $M < 2^n$. Budući da je $a + b < 2M < 2^{n - 1}$ za $a, b < M$, predlažemo da $n$ bude najmanji broj takav da važi $M < 2^{n - 1}$. Ovo obezbeđuje da se $a + b$ može zapisati sa $n$ bitova, pa ne dolazi do prekoračenja.

In [6]:
%%qsharp

operation moduloAdd(a : Qubit[], b : Qubit[], m : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(a);

    use t1 = Qubit();
    use t2 = Qubit();

    IncByLE(a, b + [t1]);
    Adjoint IncByLE(m, b + [t1]);
    CNOT(t1, t2);
    Controlled IncByLE([t2], (m, b + [t1]));
    Adjoint IncByLE(a, b + [t1]);
    X(t1);
    CNOT(t1, t2);
    X(t1);
    IncByLE(a, b + [t1]);
}

use a = Qubit[5];
use b = Qubit[5];
use m = Qubit[5];

// a = 3
// b = 3
// m = 7

X(a[0]); X(a[1]);
X(b[0]); X(b[1]);
X(m[0]); X(m[1]); X(m[2]);

moduloAdd(a, b, m);

DumpMachine();

mutable k = 0;

for i in 0..4 {
    k *= 2;
    if M(b[4 - i]) == One {
        k += 1;
    }
}

Message($"{k}");

<table class="qs-stateTable">
  <style>
    .qs-stateTable thead tr {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
    .qs-stateTable th {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody {
      pointer-events: none;
    }
    .qs-stateTable tbody td {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody td span {
      display: inline-block;
    }
    .qs-stateTable tbody tr:nth-child(even) {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
  </style>
  <thead>
    <tr>
      <th>Basis State<br />(|𝜓₁…𝜓ₙ⟩)</th>
      <th>Amplitude</th>
      <th>Measurement Probability</th>
      <th colspan="2">Phase</th>
    </tr>
  </thead>
  <tbody>
    <tr>
  <td>
    <span>|110000110011100⟩</span>
  </td>
  <td>
    <span>1.0000+0.0000𝑖</span>
  </td>
  <td>
    <progress max="100" value="100"></progress>
    <span>100.0000%</span>
  </td>
  <td style="transform: rotate(0.0000rad)">↑</td>
  <td>
    <span>0.0000</span>
  </td>
</tr>

  </tbody>
</table>


$|\psi\rangle = |110000110011100\rangle$

6

### Moduarno množenje na kvantnom računaru



Osnovna ideja ovog algoritma je da se on svede na modularno sabiranje. Pošto funkcija koju smo implementirali iznad ne računa zapravo $x +_M y$ za sve $x, y$, već samo za $x, y < M$, moramo ovde paziti da se ne desi da neki argument sabiranja prekorači $M$. Formula za množenje dva binarna broja $x$ i $y$ dužine $n$ je $x \cdot y = \sum_{i = 0}^{n - 1} \sum_{j = 0}^{n - 1} 2^{i + j} a[i] \cdot b[j]$. Svaki od sabiraka se može zameniti sa $2^{i + j} \{a[i] = 1 \land b[i] = 1\}$. Dakle, dovoljno je sračunati $2^{k}$ za $0 \leq k \leq 2n - 2$ da bi se izračunalo množenje.

Stepene dvojke po modulu broja $M$ sada možemo računati na sledeći način:
\begin{align*}
2^0 \equiv_M 1, \\
2^{k + 1} \equiv_M 2^{k} +_M 2^{k}.
\end{align*}

#### Modularno množenje na kvantnom računaru

Glavna ideja je primetiti da se proizvod $a$ i $b$ može izračunati tako što se u sumu uključe članovi $2^i b$ za koje je $a[i] = 1$. Dakle, ako znamo kako da pomnožimo broj sa $2$ po modulu $m$, onda možemo kontrolisanim operacijama sabiranja implementirati množenje. Najbolje se može razumeti implementacija iz koda.

In [7]:
%%qsharp

operation ShiftRight(a : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(a);
    // abcd --> dabc
    // ovo možemo uraditi preko SWAP operacija
    // abcd --> dbca --> dacb --> dabc
    for i in 0..n-2 {
        SWAP(a[i], a[n - 1]);
    }
    
}

// pretpostavljamo da je a < m
// n je najmanji broj takav da je m < 2^{n - 1}
// dakle, 2m < 2^n
// odavde je 2a < 2m < 2^n
// što znači da se 2a može zapisati sa n bitova
// primetimo da je a < m < 2^{n - 1}, pa poslednji bit od a ne može biti 1

operation moduloMulByTwo(a : Qubit[], m : Qubit[]) : Unit is Adj + Ctl {
    use t1 = Qubit();
    use t2 = Qubit();
    
    // prvo šiftujemo za 1 desno
    // pošto je poslednji bit 0, onda je moguće ovo izvesti preko ShiftRight operacije
    ShiftRight(a + [t1]);

    // sada je potrebno oduzeti m ako je 2a > m

    Adjoint IncByLE(m, a + [t1]);
    CNOT(t1, t2);
    Controlled IncByLE([t2], (m, a + [t1]));
    Adjoint ShiftRight(a + [t1]);
    X(t1);
    CNOT(t1, t2);
    X(t1);
    ShiftRight(a + [t1]);
}

operation moduloMultiply(a : Qubit[], b : Qubit[], r : Qubit[], m : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(a);

    // u t smeštamo 2^i b
    use t = Qubit[n];

    for i in 0..n-1 {
        CNOT(b[i], t[i]);
    }

    Controlled moduloAdd([a[0]], (t, r, m));

    for i in 1..n-1 {
        moduloMulByTwo(t, m);
        Controlled moduloAdd([a[i]], (t, r, m));
    }

    for i in 1..n-1 {
        Adjoint moduloMulByTwo(t, m);
    }

    for i in 0..n-1 {
        CNOT(b[i], t[i]);
    }
    
}

use r = Qubit[5];

moduloMultiply(a, b, r, m);

// pošto je a = 3, a b = 6 zbog prethodnog sabiranja 3 + 3 (mod 7)
// onda je a * b = 18 = 4 (mod 7)
// očekujemo r = 4, odnosno r = 00100
DumpMachine();

<table class="qs-stateTable">
  <style>
    .qs-stateTable thead tr {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
    .qs-stateTable th {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody {
      pointer-events: none;
    }
    .qs-stateTable tbody td {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody td span {
      display: inline-block;
    }
    .qs-stateTable tbody tr:nth-child(even) {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
  </style>
  <thead>
    <tr>
      <th>Basis State<br />(|𝜓₁…𝜓ₙ⟩)</th>
      <th>Amplitude</th>
      <th>Measurement Probability</th>
      <th colspan="2">Phase</th>
    </tr>
  </thead>
  <tbody>
    <tr>
  <td>
    <span>|11000011001110000100⟩</span>
  </td>
  <td>
    <span>1.0000+0.0000𝑖</span>
  </td>
  <td>
    <progress max="100" value="100"></progress>
    <span>100.0000%</span>
  </td>
  <td style="transform: rotate(0.0000rad)">↑</td>
  <td>
    <span>0.0000</span>
  </td>
</tr>

  </tbody>
</table>


$|\psi\rangle = |11000011001110000100\rangle$

### Modularno stepenovanje na kvantnom računaru

Primetimo najpre da važi sledeća jednakost:
\begin{align*}
    a^x = a^{\sum_{i = 0}^{n - 1} 2^i x_i} = \prod_{i = 0, x_i = 1}^{i = n - 1} a^{2^i}
\end{align*}

Ovaj proizvod se može sračunati iterativno. Najpre je on $1$, pa ga množimo sa $a^2$, pa onda sa $(a^2)^2 = a^{2^2}$, itd. Dakle, ako napravimo funkciju koja će računati kvadrat broja, možemo implementirati stepenovanje.

In [8]:
%%qsharp

operation moduloSquare(a : Qubit[], r : Qubit[], m : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(a);
    use t = Qubit[n];

    within {
        for i in 0..n-1 {
            CNOT(a[i], t[i]);
        }
    } apply {
        moduloMultiply(a, t, r, m);
    }
}

operation Copy(a : Qubit[], r : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(a);
    for i in 0..n-1 {
        CNOT(a[i], r[i]);
    }
}

operation moduloExp(a : Qubit[], x : Qubit[], r : Qubit[], m : Qubit[]) : Unit is Adj + Ctl {
    let n = Length(a);
    let k = Length(x);

    // generišemo niz u koji ćemo da smestimo a^{2^i}
    use temp = Qubit[n*k];
    mutable t = Chunks(n, temp);

    use p = Qubit[n * (k - 1)];
    mutable partial = Chunks(n, p);

    within {
        Copy(a, t[0]);

        for i in 1..k-1 {
            moduloSquare(t[i - 1], t[i], m);
        }

        Controlled Copy([x[0]], (a, partial[0]));
        X(x[0]);
        CNOT(x[0], partial[0][0]);
        X(x[0]);

        for i in 1..k-2 {
            Controlled moduloMultiply([x[i]], (t[i], partial[i - 1], partial[i], m));
            X(x[i]);
            Controlled Copy([x[i]], (partial[i - 1], partial[i]));
            X(x[i]);
        }

    } apply {
        Controlled moduloMultiply([x[k - 1]], (t[k - 1], partial[k - 2], r, m));
        X(x[k - 1]);
        Controlled Copy([x[k - 1]], (partial[k - 2], r));
        X(x[k - 1]);
    }
}

for i in 0..4 {
    Reset(r[i]);
    Reset(a[i]);
    Reset(b[i]);
}

// a = 5
X(a[0]); X(a[2]);
// b = 2
X(b[1]);

moduloExp(a, b, r, m);

// očekujemo r = 5^2 = 25 = 4 (mod 7) = 00100

DumpMachine();

for i in 0..4 {
    Reset(r[i]);
}

// postavimo sada b = 3
X(b[0]);

moduloExp(a, b, r, m);

// očekujemo 4 * 5 = 20 = 6 (mod 7) = 01100

DumpMachine();

<table class="qs-stateTable">
  <style>
    .qs-stateTable thead tr {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
    .qs-stateTable th {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody {
      pointer-events: none;
    }
    .qs-stateTable tbody td {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody td span {
      display: inline-block;
    }
    .qs-stateTable tbody tr:nth-child(even) {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
  </style>
  <thead>
    <tr>
      <th>Basis State<br />(|𝜓₁…𝜓ₙ⟩)</th>
      <th>Amplitude</th>
      <th>Measurement Probability</th>
      <th colspan="2">Phase</th>
    </tr>
  </thead>
  <tbody>
    <tr>
  <td>
    <span>|10100010001110000100⟩</span>
  </td>
  <td>
    <span>1.0000+0.0000𝑖</span>
  </td>
  <td>
    <progress max="100" value="100"></progress>
    <span>100.0000%</span>
  </td>
  <td style="transform: rotate(0.0000rad)">↑</td>
  <td>
    <span>0.0000</span>
  </td>
</tr>

  </tbody>
</table>


$|\psi\rangle = |10100010001110000100\rangle$

<table class="qs-stateTable">
  <style>
    .qs-stateTable thead tr {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
    .qs-stateTable th {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody {
      pointer-events: none;
    }
    .qs-stateTable tbody td {
      text-align: left;
      border: none;
    }
    .qs-stateTable tbody td span {
      display: inline-block;
    }
    .qs-stateTable tbody tr:nth-child(even) {
      background-color: var(
        --vscode-list-hoverBackground,
        var(--jp-layout-color1, inherit)
      );
    }
  </style>
  <thead>
    <tr>
      <th>Basis State<br />(|𝜓₁…𝜓ₙ⟩)</th>
      <th>Amplitude</th>
      <th>Measurement Probability</th>
      <th colspan="2">Phase</th>
    </tr>
  </thead>
  <tbody>
    <tr>
  <td>
    <span>|10100110001110001100⟩</span>
  </td>
  <td>
    <span>1.0000+0.0000𝑖</span>
  </td>
  <td>
    <progress max="100" value="100"></progress>
    <span>100.0000%</span>
  </td>
  <td style="transform: rotate(0.0000rad)">↑</td>
  <td>
    <span>0.0000</span>
  </td>
</tr>

  </tbody>
</table>


$|\psi\rangle = |10100110001110001100\rangle$